In [ ]:
import pandas as pd
import random
from time import time

In [ ]:
random.seed(42)

In [ ]:
N_MINUTES = 10
# log_search = True
verbose = 1

In [ ]:
distance_matrix = pd.read_csv('/kaggle/input/santa-2021/distance_matrix.csv', index_col='Permutation')
permutations = pd.read_csv('/kaggle/input/santa-2021/permutations.csv')

In [ ]:
s1_options = []
s2_options = []
s3_options = []
all_options = []
for permutation in permutations['Permutation']:
    if permutation.startswith('🎅🤶'):
        s1_options.append(permutation)
        s2_options.append(permutation)
        s3_options.append(permutation)
    else:
        all_options.append(permutation)

In [ ]:
class String:
    def __init__(self, s, cost):
        self.s = s
        self.cost = cost

In [ ]:
s1 = s1_options.copy()
s2 = s2_options.copy()
s3 = s3_options.copy()

for option in all_options:
    index = random.randint(0, 2)
    if index == 0:
        s1.append(option)
    elif index == 1:
        s2.append(option)
    else:
        s3.append(option)

In [ ]:
def calculate_cost(s):
    cost = sum([distance_matrix.loc[s[i], s[i+1]] for i in range(len(s)-1)])
    cost += 14
    cost -= distance_matrix.loc[s[0], s[1]]
    cost -= distance_matrix.loc[s[-2], s[-1]]
    
    return cost

In [ ]:
cost1 = calculate_cost(s1)
cost2 = calculate_cost(s2)
cost3 = calculate_cost(s3)

In [ ]:
strings = [
    String(s1, cost1),
    String(s2, cost2),
    String(s3, cost3),
]

# Local Search

In [ ]:
def swap_in_string(string):
    attempts = 0
    while attempts < 5:
        two_indices = random.randint(0, len(string.s)-1), random.randint(0, len(string.s)-1)

        new_cost = string.cost

        if two_indices[0] != 0:
            new_cost -= distance_matrix.loc[string.s[two_indices[0]-1], string.s[two_indices[0]]]
        if two_indices[0] != len(string.s)-1:
            new_cost -= distance_matrix.loc[string.s[two_indices[0]], string.s[two_indices[0]+1]]

        if two_indices[1] != 0:
            new_cost -= distance_matrix.loc[string.s[two_indices[1]-1], string.s[two_indices[1]]]
        if two_indices[1] != len(string.s)-1:
            new_cost -= distance_matrix.loc[string.s[two_indices[1]], string.s[two_indices[1]+1]]
            
        string.s[two_indices[0]], string.s[two_indices[1]] = string.s[two_indices[1]], string.s[two_indices[0]]
            
        if two_indices[0] != 0:
            new_cost += distance_matrix.loc[string.s[two_indices[0]-1], string.s[two_indices[0]]]
        if two_indices[0] != len(string.s)-1:
            new_cost += distance_matrix.loc[string.s[two_indices[0]], string.s[two_indices[0]+1]]

        if two_indices[1] != 0:
            new_cost += distance_matrix.loc[string.s[two_indices[1]-1], string.s[two_indices[1]]]
        if two_indices[1] != len(string.s)-1:
            new_cost += distance_matrix.loc[string.s[two_indices[1]], string.s[two_indices[1]+1]]
        
        if new_cost >= string.cost:
            string.s[two_indices[0]], string.s[two_indices[1]] = string.s[two_indices[1]], string.s[two_indices[0]]
        if new_cost < string.cost:
            string.cost = new_cost
            return string, True
        
        attempts += 1
        
    return string, False

In [ ]:
def swap_bw_strings(string1, string2):
    attempts = 0
    while attempts < 10:
        two_indices = random.randint(0, len(string1.s)-1), random.randint(0, len(string2.s)-1)
        
        if string1.s[two_indices[0]] not in all_options or string2.s[two_indices[1]] not in all_options:
            continue

        new_cost_1 = string1.cost
        new_cost_2 = string2.cost

        if two_indices[0] != 0:
            new_cost_1 -= distance_matrix.loc[string1.s[two_indices[0]-1], string1.s[two_indices[0]]]
        if two_indices[0] != len(string1.s)-1:
            new_cost_1 -= distance_matrix.loc[string1.s[two_indices[0]], string1.s[two_indices[0]+1]]
        if two_indices[0] != 0:
            new_cost_1 += distance_matrix.loc[string1.s[two_indices[0]-1], string2.s[two_indices[1]]]
        if two_indices[0] != len(string1.s)-1:
            new_cost_1 += distance_matrix.loc[string2.s[two_indices[1]], string1.s[two_indices[0]+1]]

        if two_indices[1] != 0:
            new_cost_2 -= distance_matrix.loc[string2.s[two_indices[1]-1], string2.s[two_indices[1]]]
        if two_indices[1] != len(string2.s)-1:
            new_cost_2 -= distance_matrix.loc[string2.s[two_indices[1]], string2.s[two_indices[1]+1]]
        if two_indices[1] != 0:
            new_cost_2 += distance_matrix.loc[string2.s[two_indices[1]-1], string1.s[two_indices[0]]]
        if two_indices[1] != len(string2.s)-1:
            new_cost_2 += distance_matrix.loc[string1.s[two_indices[0]], string2.s[two_indices[1]+1]]

        if sum((new_cost_1, new_cost_2)) < sum((string1.cost, string2.cost)):
            string1.s[two_indices[0]], string2.s[two_indices[1]] = string2.s[two_indices[1]], string1.s[two_indices[0]]
            string1.cost = new_cost_1
            string2.cost = new_cost_2
            return string1, string2, True
        
        attempts += 1
        
    return string1, string2, False

In [ ]:
def transfer_bw_strings(string1, string2):
    attempts = 0
    while attempts < 10:
        two_indices = random.randint(0, len(string1.s)-1), random.randint(0, len(string2.s)-1)
        
        if string1.s[two_indices[0]] not in all_options:
            continue

        new_cost_1 = string1.cost
        new_cost_2 = string2.cost

        if two_indices[0] != 0:
            new_cost_1 -= distance_matrix.loc[string1.s[two_indices[0]-1], string1.s[two_indices[0]]]
        if two_indices[0] != len(string1.s)-1:
            new_cost_1 -= distance_matrix.loc[string1.s[two_indices[0]], string1.s[two_indices[0]+1]]
        if two_indices[0] != 0 and two_indices[0] != len(string1.s)-1:
            new_cost_1 += distance_matrix.loc[string1.s[two_indices[0]-1], string1.s[two_indices[0]+1]]

        if two_indices[1] != 0:
            new_cost_2 -= distance_matrix.loc[string2.s[two_indices[1]-1], string2.s[two_indices[1]]]
        if two_indices[1] != 0:
            new_cost_2 += distance_matrix.loc[string2.s[two_indices[1]-1], string1.s[two_indices[0]]]
        new_cost_2 += distance_matrix.loc[string1.s[two_indices[0]], string2.s[two_indices[1]]]

        if sum((new_cost_1, new_cost_2)) < sum((string1.cost, string2.cost)):
            string2.s.insert(two_indices[1], string1.s[two_indices[0]])
            string1.s.pop(two_indices[0])
            string1.cost = new_cost_1
            string2.cost = new_cost_2
            return string1, string2, True
        
        attempts += 1
        
    return string1, string2, False

In [ ]:
start = time()

if verbose != 0:
    print('Starting Search')
    print(f'Initial Cost: s1={strings[0].cost}, s2={strings[1].cost}, s3={strings[2].cost}, total={max(strings[0].cost, strings[1].cost, strings[2].cost)}')
loop_count = 0
while time()-start <= N_MINUTES*60:
    loop_count += 1
    option = random.randint(0, 2)
    if option == 0:
        index = random.randint(0, 2)
        strings[index], updated = swap_in_string(strings[index])
    elif option == 1:
        index1, index2 = random.sample([0, 1, 2], k=2)
        strings[index1], strings[index2], updated = swap_bw_strings(strings[index1], strings[index2])
    elif option == 2:
        index1, index2 = random.sample([0, 1, 2], k=2)
        strings[index1], strings[index2], updated = transfer_bw_strings(strings[index1], strings[index2])
                
    if not updated:
        continue
    if verbose == 1 and loop_count % 20 == 0:
        print(f'Cost: s1={strings[0].cost}, s2={strings[1].cost}, s3={strings[2].cost}, total={max(strings[0].cost, strings[1].cost, strings[2].cost)}')
    elif verbose == 2:
        print(f'Cost: s1={strings[0].cost}, s2={strings[1].cost}, s3={strings[2].cost}, total={max(strings[0].cost, strings[1].cost, strings[2].cost)}')

In [ ]:
def create_result(s):
    result = s[0]
    for permutation in s[1:]:
        joined = False
        for index in range(1, len(permutation)+1):
            if permutation[:index] == result[-index:]:
                assert len(permutation[index:]) == distance_matrix.loc[result[-7:], permutation], f'{result}\n{permutation}'
                result += permutation[index:]
                joined = True
                break
        if not joined:
            result += permutation
        
    return result

In [ ]:
result1 = create_result(s1)
result2 = create_result(s2)
result3 = create_result(s3)

In [ ]:
submission = pd.Series(
    [result1,
     result2,
     result3],
    name='schedule'
)

submission

In [ ]:
# Evaluating score

max(map(len, submission.to_list()))

In [ ]:
submission.to_csv('submission.csv', index=False)